In [207]:
import numpy as np
import matplotlib.pyplot as plt
import functions
import math
import numdifftools as nd


In [208]:
# %load functions.py


def test_function_1var(x):
    return x**2+1
def test_function_2var(x):
    return x[0]**3+3*x[1]+10

def quartic(x):
    return x**4 - 3*x**2 + 2*x + 1

def fifth_degree(x):
    return 2*x**5 - 5*x**4 + 3*x**3 + x**2 - 2*x + 1

def exponential_plus_c(x):
    c = 3
    return math.exp(x) + c

def rational(x):
    return (x**3 - x**2 + 1)/(x - 1)

def third_degree(x):
    return -7*x**2+0.5*x-7

def fun_2vars(x):
    return 0.5*(x[0] - 4.5)**2 + 2.5*(x[1] - 2.3)**2

def cos_fun(x):
    return np.cos(x)-2*x**3

<h1>Implementation of Steepest Descent</h1>

In [209]:
#some = test_function_1var(1)
grad = nd.Gradient(test_function_1var)([2])
print(np.round(grad))

grad = nd.Gradient(test_function_2var)([2,2])
print(grad)




4.0
[12.  3.]


In [260]:
def line_search(function,step, x, gradient_x, ro, c = 1e-4, tol = 1e-8):
    '''
    Inexact line search where the step length is updated through the Armijo condition:
    $ f (x_k + α * p_k ) ≤ f ( x_k ) + c * α * ∇ f_k^T * p_k $

    Args:
      - step: starting alpha value
      - x: current point
      - gradient_x: gradient of the current point
      - c: constant value; default: 1e-4
      - tol: tolerance value   
    Out:
      - New value of step: the first value found respecting the Armijo condition
    '''
    f_x = function(x)
    gradient_square_norm = np.linalg.norm(gradient_x)**2
    # Until the sufficient decrease condition is met 
    while function(x - step * gradient_x) >= (f_x - c * step * gradient_square_norm):
        
        # Update the stepsize (backtracking)
        step = ro*step
        # If the step size falls below a certain tolerance, exit the loop
        if step < tol:
            break
    return step


def steepest_descent(function, x0, tolerance, ro, max_iter = 10000): 
    '''
    Steepest descent with alpha updated through line search (Armijo condition).
    
    Args:
      - function:objective function
      - x0: initial guess for x_0 and x_1 (default values: zero) <numpy.ndarray>
      - max_iter: maximum number of iterations (default: 10000)
      - tolerance: minimum gradient magnitude at which the algorithm stops (default: 1e-6)
    
    Out:
      - results: <numpy.ndarray> of size (n_iter, 2) with x_0 and x_1 values at each iteration
    '''
    
    # Initialize the iteration counter and the step size 
    iter_count = 1
    # Prepare list to store results at each iteration 
    results = np.array([])
    # Evaluate the gradient at the starting point 
    gradient_x = nd.Gradient(function)(x0)
    #gradient_x = gradient(x0)
    # print(np.absolute(gradient_x))
    # set the initial point 
    x = x0 
    results = np.append(results, x, axis=0)
   
    if len(x0)>1:
        criterion = any(np.absolute(gradient_x) > tolerance)
    else:  
        criterion = np.absolute(gradient_x)>tolerance 
    # Iterate until the gradient is below the tolerance or maximum number of iterations is reached
    # Stopping criterion: inf norm of the gradient (max abs)
    while criterion and iter_count<max_iter:

        # Update the step size through the Armijo condition
        # Note: the first value of alpha is commonly set to 1
        # print(gradient_x)
        # print(x)
        if len(x)>1:
            alpha = line_search(function,1, x, gradient_x,ro)
        else:
            alpha = line_search(function,1, x[0], gradient_x,ro)
        print('New alpha',alpha)

        # Update the current point by moving in the direction of the negative gradient 
        x = x - alpha * gradient_x
        # Store the result
        results = np.append(results, x, axis=0)
        
        # Evaluate the gradient at the new point 
        # gradient_x = gradient(x) 
        gradient_x = nd.Gradient(function)(x)
                
        # Increment the iteration counter 
        iter_count += 1 
        print(f'Iteration: {iter_count} , x = {str(x)} , f(x)={function(x)}')
    # Return the points obtained at each iteration
    return results.reshape(-1, 2)

In [261]:
def_x0 = [1]
tolerance = 1e-6 
ro = 0.5
# Steepest descent
estimate = steepest_descent(
  cos_fun, x0 = def_x0, tolerance=tolerance, ro=ro,max_iter=3000)

# Print results
# print(estimate)
# print('- Final results: {}'.format(estimate[-1]))
# print('- N° iterations: {}'.format(len(estimate)))

# # # Steepest descent steps
# #X_estimate= estimate[:, 0]
# Z_estimate = rational(estimate[-1])

# print(Z_estimate)

New alpha 1
Iteration: 2 , x = [7.84147098] , f(x)=[-964.31068925]
New alpha 1
Iteration: 3 , x = [377.77339596] , f(x)=[-1.07826151e+08]
New alpha 1
Iteration: 4 , x = [856654.91046935] , f(x)=[-1.25732549e+18]
New alpha 1
Iteration: 5 , x = [4.40314667e+12] , f(x)=[-1.70733779e+38]
New alpha 1
Iteration: 6 , x = [1.1632487e+26] , f(x)=[-3.14809425e+78]
New alpha 7.450580596923828e-09
Iteration: 7 , x = [1.1632487e+26] , f(x)=[-3.14809425e+78]
New alpha 7.450580596923828e-09
Iteration: 8 , x = [1.1632487e+26] , f(x)=[-3.14809425e+78]
New alpha 7.450580596923828e-09
Iteration: 9 , x = [1.1632487e+26] , f(x)=[-3.14809425e+78]
New alpha 7.450580596923828e-09
Iteration: 10 , x = [1.1632487e+26] , f(x)=[-3.14809425e+78]
New alpha 7.450580596923828e-09
Iteration: 11 , x = [1.1632487e+26] , f(x)=[-3.14809425e+78]
New alpha 7.450580596923828e-09
Iteration: 12 , x = [1.1632487e+26] , f(x)=[-3.14809425e+78]
New alpha 7.450580596923828e-09
Iteration: 13 , x = [1.1632487e+26] , f(x)=[-3.14809425e

<H1>Newton Method implementation</H1>

In [254]:
def newton_method(function, x0, tolerance, max_iter = 10000): 
    iter_count = 1
    # Prepare list to store results at each iteration 
    results = np.array([])
    # Iterate until the gradient is below the tolerance or maximum number of iterations is reached
    # Stopping criterion: inf norm of the gradient (max abs)
    x_l = x0
    # while np.absolute(nd.Gradient(function)(x_l))<=tolerance  or iter_count<max_iter:
    #testing critetion abs(function(x_l[0]))>tolerance  or iter_count>max_iter
    while np.absolute(nd.Gradient(function)(x_l))>tolerance or iter_count>max_iter:
        iter_count += 1 
        x = x_l-function(x_l[0])/nd.Gradient(function)(x_l)
        results = np.append(results, x, axis=0)
        print(f'Iteration: {iter_count} , x = {str(x)} , f(x)={function(x)}')
        x_l = x
        
    return results.reshape(-1, 2)

In [255]:
newton_method(cos_fun,[1],1E-15)

6.841470984807919
Iteration: 2 , x = [0.78663979] , f(x)=[-0.26732052]
4.42079714174927
Iteration: 3 , x = [0.72617094] , f(x)=[-0.01813265]
3.8279568229636247
Iteration: 4 , x = [0.72143404] , f(x)=[-0.00010595]
3.783264548531323
Iteration: 5 , x = [0.72140603] , f(x)=[-3.6893425e-09]
3.7830010763184467
Iteration: 6 , x = [0.72140603] , f(x)=[-1.11022302e-16]


ValueError: cannot reshape array of size 5 into shape (2)